In [133]:
#Import libraries
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from matplotlib.ticker import PercentFormatter
from operator import attrgetter
import matplotlib.colors as mcolors
warnings.filterwarnings("ignore")
import matplotlib.ticker as tkr

#Use fivethirtyeight style
plt.style.use('fivethirtyeight')

#Cancel any style and reset matplotlib styles
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 5]

In [134]:
fname = 'C:\\tmp\\workdir\\BBdata\\GitHub\\Climate.Change.Project2\\Data\\Dataset_Climate.xlsx'
df = pd.read_excel(fname)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   LineNo            8950 non-null   int64 
 1   DisNo             8950 non-null   object
 2   Year              8950 non-null   int64 
 3   DisasterSubgroup  8950 non-null   object
 4   DisasterType      8950 non-null   object
 5   DisasterSubtype   7655 non-null   object
 6   Country           8950 non-null   object
 7   ISO               8950 non-null   object
 8   Region            8950 non-null   object
 9   Continent         8950 non-null   object
 10  TotalDamages      8950 non-null   int64 
dtypes: int64(3), object(8)
memory usage: 769.3+ KB


,LineNo,DisNo,Year,DisasterSubgroup,DisasterType,DisasterSubtype,Country,ISO,Region,Continent,TotalDamages
0,1,2000-0372-CHN,2000,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,19
1,2,2000-9186-AFG,2000,Climatological,Drought,Drought,Afghanistan,AFG,Southern Asia,Asia,326
2,3,2000-0373-BGD,2000,Hydrological,Flood,Flash flood,Bangladesh,BGD,Southern Asia,Asia,145
3,4,2000-0905-BOL,2000,Hydrological,Flood,NaN,Bolivia (Plurinational State of),BOL,South America,Americas,145
4,5,2000-0019-BRA,2000,Hydrological,Flood,Riverine flood,Brazil,BRA,South America,Americas,145


In [135]:
df['Recency'] = (max(df['Year']) - df['Year'])
df.head()

,LineNo,DisNo,Year,DisasterSubgroup,DisasterType,DisasterSubtype,Country,ISO,Region,Continent,TotalDamages,Recency
0,1,2000-0372-CHN,2000,Hydrological,Flood,Riverine flood,China,CHN,Eastern Asia,Asia,19,21
1,2,2000-9186-AFG,2000,Climatological,Drought,Drought,Afghanistan,AFG,Southern Asia,Asia,326,21
2,3,2000-0373-BGD,2000,Hydrological,Flood,Flash flood,Bangladesh,BGD,Southern Asia,Asia,145,21
3,4,2000-0905-BOL,2000,Hydrological,Flood,NaN,Bolivia (Plurinational State of),BOL,South America,Americas,145,21
4,5,2000-0019-BRA,2000,Hydrological,Flood,Riverine flood,Brazil,BRA,South America,Americas,145,21


In [170]:
df_rfm = df.groupby('Country').agg({'Recency':'min'
                                           ,'Year':pd.Series.nunique
                                           ,'TotalDamages':'sum'})
df_rfm.head(20)


,Recency,Year,TotalDamages
Country,,,
Afghanistan,0,22,33522
Albania,0,13,6301
Algeria,0,19,18834
American Samoa,12,4,1727
Angola,0,21,7560
Anguilla,4,1,200
Antigua and Barbuda,4,3,839
Argentina,0,22,23145
Armenia,1,7,2146


In [171]:
df_rfm.rename(columns={'Year':'Frequency','TotalDamages':'Monetary'},inplace=True)

In [172]:
df_rfm

,Recency,Frequency,Monetary
Country,,,
Afghanistan,0,22,33522
Albania,0,13,6301
Algeria,0,19,18834
American Samoa,12,4,1727
Angola,0,21,7560
...,...,...,...
Virgin Island (U.S.),4,2,1728
Wallis and Futuna,9,1,576
Yemen,0,18,7322


In [173]:
df_rfm["recency_score"] = pd.qcut(df_rfm['Recency'], q = 8, labels=[4, 3, 2, 1], duplicates = 'drop')
df_rfm["recency_score"] = df_rfm.recency_score.astype(int)

In [174]:
df_rfm["frequency_score"] = pd.qcut(df_rfm['Frequency'], q = 4, labels=[4, 3, 2, 1], duplicates = 'drop')
df_rfm["frequency_score"] = df_rfm.frequency_score.astype(int)


In [175]:
df_rfm["monetary_score"] = pd.qcut(df_rfm['Monetary'], q = 4, labels=[4, 3, 2, 1], duplicates = 'drop')
df_rfm["monetary_score"] = df_rfm.monetary_score.astype(int)

In [176]:
df_rfm["RFM_Segment"] = df_rfm.recency_score * 100 + df_rfm.frequency_score * 10 + df_rfm.monetary_score

In [177]:
df_rfm["RFM_Score"] = df_rfm.recency_score + df_rfm.frequency_score  + df_rfm.monetary_score

In [178]:
df_rfm

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score
Country,,,,,,,,
Afghanistan,0,22,33522,4,1,1,411,6
Albania,0,13,6301,4,3,2,432,9
Algeria,0,19,18834,4,2,1,421,7
American Samoa,12,4,1727,1,4,4,144,9
Angola,0,21,7560,4,1,2,412,7
...,...,...,...,...,...,...,...,...
Virgin Island (U.S.),4,2,1728,2,4,4,244,10
Wallis and Futuna,9,1,576,1,4,4,144,9
Yemen,0,18,7322,4,2,2,422,8


In [180]:
d1 = pd.Series(df_rfm["RFM_Segment"].unique())
d1.head(20)

0     411
1     432
2     421
3     144
4     412
5     244
6     433
7     143
8     332
9     334
10    422
11    423
12    443
13    333
14    234
15    434
16    142
17    431
18    424
19    243
dtype: int32

In [192]:
segt_map = {
    r'[1-2][1-2][1-2]': 'Low',
    r'[1-3][1-2][1-2]': 'Low',
    r'[1-2][1-3][1-2]': 'Low',
    r'[1-2][1-2][1-3]': 'Low',
    r'[1-2][1-2][1-2]': 'Low',
    r'[1-4][1-2][1-2]': 'Low',
    r'[1-2][1-4][1-2]': 'Low',
    r'[1-2][1-2][1-4]': 'Low',

    r'[2-3][1-3][1-3]': 'Medium',
    r'[2-3][1-3][1-3]': 'Medium',
    r'[2-2][1-3][1-3]': 'Medium',
    r'[2-2][1-3][1-4]': 'Medium',
    r'[2-2][1-3][1-3]': 'Medium',
    r'[2-4][1-3][1-3]': 'Medium',
    r'[2-2][1-4][1-3]': 'Medium',
    r'[2-2][1-3][1-4]': 'Medium',

    r'[1-4][1-3][1-4]': 'High',
    r'[1-4][1-3][1-4]': 'High',
    r'[1-2][1-4][1-4]': 'High',
    r'[1-4][1-4][1-4]': 'High',
    r'[1-4][1-4][1-4]': 'High',
    r'[1-4][1-4][1-4]': 'High',
    r'[1-4][1-4][1-3]': 'High',
    r'[1-4][1-4][1-4]': 'High'   
}
df_rfm['Segment_labels'] = df_rfm['RFM_Segment']
df_rfm['Segment_labels'] = df_rfm['Segment_labels'].astype(str).replace(segt_map, regex=True)



In [193]:
df_rfm

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score,Segment_labels
Country,,,,,,,,,
Afghanistan,0,22,33522,4,1,1,411,6,Low
Albania,0,13,6301,4,3,2,432,9,Medium
Algeria,0,19,18834,4,2,1,421,7,Low
American Samoa,12,4,1727,1,4,4,144,9,High
Angola,0,21,7560,4,1,2,412,7,Low
...,...,...,...,...,...,...,...,...,...
Virgin Island (U.S.),4,2,1728,2,4,4,244,10,High
Wallis and Futuna,9,1,576,1,4,4,144,9,High
Yemen,0,18,7322,4,2,2,422,8,Low


In [196]:
df_rfm[ df_rfm['Segment_labels'] == 'Low']


,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_Segment,RFM_Score,Segment_labels
Country,,,,,,,,,
Afghanistan,0,22,33522,4,1,1,411,6,Low
Algeria,0,19,18834,4,2,1,421,7,Low
Angola,0,21,7560,4,1,2,412,7,Low
Argentina,0,22,23145,4,1,1,411,6,Low
Australia,0,22,53040,4,1,1,411,6,Low
...,...,...,...,...,...,...,...,...,...
Uruguay,1,15,4736,4,2,2,422,8,Low
Venezuela (Bolivarian Republic of),0,17,4737,4,2,2,422,8,Low
Viet Nam,0,22,51723,4,1,1,411,6,Low


In [198]:
df_rfm.to_excel('C:\\tmp\\test1.xlsx')